In [115]:
import pandas as pd
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from newspaper import Article, fulltext
from tqdm.auto import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
import seaborn as sns
from matplotlib import pyplot as plt
tqdm.pandas()
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import sys
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np
from sklearn.linear_model import SGDClassifier
from collections import Counter
import re
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
import wordninja

from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home2/pygenest/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home2/pygenest/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load data

In [2]:
train_labels = pd.read_json('Hackathon_data/train/D1_train-truth.jsonl', orient='records', lines=True).set_index('uid', drop=True).sort_index()['label']
train_data = pd.read_json('Hackathon_data/train/D1_train.jsonl', orient='records', lines=True).set_index('uid', drop=True).sort_index()
train_data['label'] = train_labels

train_data['html_soup'] = train_data['html'].progress_apply(lambda html: BeautifulSoup(html, 'lxml'))

  1%|          | 157/19783 [00:02<04:47, 68.19it/s]/tmp/ipykernel_3426186/2087471578.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  train_data['html_soup'] = train_data['html'].progress_apply(lambda html: BeautifulSoup(html, 'lxml'))
  1%|          | 226/19783 [00:04<07:59, 40.76it/s]/home2/pygenest/miniconda3/envs/hackathon_passau/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
 61%|██████    | 12075/19783 [03:31<01:31, 84.27i

In [3]:
val_labels = pd.read_json('Hackathon_data/validation/D1_validation-truth.jsonl', orient='records', lines=True).set_index('uid', drop=True).sort_index()['label']
val_data = pd.read_json('Hackathon_data/validation/D1_validation.jsonl', orient='records', lines=True).set_index('uid', drop=True).sort_index()

val_data['html_soup'] = val_data['html'].progress_apply(lambda html: BeautifulSoup(html, 'lxml'))

 26%|██▋       | 777/2937 [00:08<00:16, 132.13it/s]/home2/pygenest/miniconda3/envs/hackathon_passau/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
 28%|██▊       | 813/2937 [00:08<00:14, 146.27it/s]/tmp/ipykernel_3426186/3243447638.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  val_data['html_soup'] = val_data['html'].progress_apply(lambda html: BeautifulSoup(html, 'lxml'))
100%|██████████| 2937/2937 [00:48<00:00, 61.03it/s] 


# Text article

In [4]:
language_translation = {
    'en': 'english',
    'de': 'german',
    'hr': 'hungarian',
    'nl': 'dutch',
    'ru': 'russian',
    'zh-cn': 'chinese',
    'pt': 'portuguese',
    'fr': 'french',
    'id': 'indonesia',
    'es': 'spanish',
    'tr': 'turkish',
    'it': 'italian'
}

In [5]:
stop_words = {}
for key, lang in language_translation.items():
    try:
        stop_words[key] = stopwords.words(lang)
    except:
        pass

In [6]:
stemmers = {}
for key, lang in language_translation.items():
    try:
        stemmers[key] = SnowballStemmer(language=lang)
    except:
        pass

In [7]:
def detect_language(text: str):
    try:
        return detect(text)
    except:
        return None
    
def preprocess_text(row) -> str:
    lang = row['lang']
    text = row['text']
    text = text.lower()
    tokens = [w for w in word_tokenize(text)]

    # Handle stopwords
    if lang in stop_words:
        tokens = [w for w in tokens if w not in stop_words[lang]]
    if lang in stemmers:
        stemmer = stemmers[lang]
        tokens = [stemmer.stem(w) for w in tokens]
    return ' '.join(tokens)

In [23]:
vectorizer = TfidfVectorizer(max_features=15000)

def preprocess_features(dataset: pd.DataFrame, train=True):
    #dataset['text'] = dataset['html_soup'].progress_apply(lambda soup: soup.text)
    #dataset['lang'] = dataset['text'].progress_apply(detect_language)
    #dataset['processed_text'] = dataset.progress_apply(preprocess_text, axis='columns')
    if train:
        tokens = vectorizer.fit_transform(dataset['processed_text'])
    else:
        tokens = vectorizer.transform(dataset['processed_text'])
    return tokens

train_features = preprocess_features(train_data, train=True)
val_features = preprocess_features(val_data, train=False)

# Train model

In [62]:
#model = RandomForestClassifier(n_estimators=500, n_jobs=-1)
model = SGDClassifier(loss='modified_huber')
#model = SVC(kernel='rbf')
#model = GradientBoostingClassifier()
model.fit(train_features, train_labels);

In [63]:
train_predictions = model.predict(train_features)
val_predictions = model.predict(val_features)

# Evaluate model

In [64]:
print(classification_report(train_labels, train_predictions))

              precision    recall  f1-score   support

       Adult       1.00      0.99      1.00      4301
      Benign       0.93      0.91      0.92      4789
   Malicious       0.96      0.97      0.96     10693

    accuracy                           0.96     19783
   macro avg       0.96      0.96      0.96     19783
weighted avg       0.96      0.96      0.96     19783



In [65]:
print(classification_report(val_labels, val_predictions))

              precision    recall  f1-score   support

       Adult       0.91      0.85      0.88        75
      Benign       0.72      0.70      0.71       805
   Malicious       0.88      0.89      0.89      2057

    accuracy                           0.84      2937
   macro avg       0.84      0.81      0.83      2937
weighted avg       0.84      0.84      0.84      2937



# URL model

In [121]:
def extract_domain_and_path(url):
    # Supprimer le protocole s'il est présent
    url = url.replace('www', '')

    # Extraire le nom de domaine et le chemin
    parts = url.split('/')
    domain = parts[0].replace('.', ' ')
    path = ' '.join(parts[1:])

    # Concaténer le nom de domaine et le chemin
    result = f"{domain} {path}"

    return result.strip()

def separate_words(input_str):
    # Utiliser wordninja pour segmenter les mots
    words = wordninja.split(input_str)

    # Joindre les mots séparés avec des espaces
    result = ' '.join(words)

    return result

def top_words(input_str, n=1000):
    input_str = [word for word in word_tokenize(input_str) if len(word) > 1]
    input_str = ' '.join(input_str)
    # Convertir le texte en minuscules et diviser en mots
    words = re.findall(r'\b\w+\b', input_str.lower())

    # Compter les occurrences des mots
    word_counts = Counter(words)

    # Afficher les n mots les plus courants
    top_n_words = word_counts.most_common(1000)
    
    L = []
    for word, count in top_n_words:
        L.append(word)
    return L
    
def tokenize_url(url, corpus):
    v = np.zeros(len(corpus) + 3, dtype=int)
    for idx, w in enumerate(corpus):
        n = url.count(w)
        if n > 0:
            v[idx] = n
    v[-1] = 1 if url.count('/') > 2 else 0
    v[-2] = 0 if sum(c.isdigit() for c in url) == 0 else 1
    v[-3] = 1 if url.count('-') > 0 else 0
    return v

In [163]:
def preprocess_url_dataset(dataset: pd.DataFrame, train=True, corpus=None):
    dataset['url_words'] = dataset['url'].apply(lambda url: separate_words(extract_domain_and_path(url)))
    if train:
        grouped_df = dataset.groupby('label')['url_words'].apply(lambda x: ' '.join(x)).reset_index()
        top_words_result_A = top_words(grouped_df['url_words'][0])
        top_words_result_B = top_words(grouped_df['url_words'][1])
        top_words_result_M = top_words(grouped_df['url_words'][2])
        corpus = list(set(top_words_result_A + top_words_result_B + top_words_result_M))

    dataset['url_vector'] = dataset['url'].apply(lambda url: tokenize_url(url, corpus))
    return dataset['url_vector'].tolist(), corpus

In [165]:
train_features4, corpus = preprocess_url_dataset(train_data, train=True)
val_features4, corpus = preprocess_url_dataset(val_data, train=False, corpus=corpus)

In [166]:
model4 = SGDClassifier(loss='modified_huber')
model4.fit(train_features4, train_labels);

In [167]:
train_predictions4 = model4.predict(train_features4)
val_predictions4 = model4.predict(val_features4)

In [168]:
print(classification_report(train_labels, train_predictions4))

              precision    recall  f1-score   support

       Adult       1.00      1.00      1.00      4301
      Benign       0.98      0.99      0.99      4789
   Malicious       1.00      0.99      1.00     10693

    accuracy                           0.99     19783
   macro avg       0.99      0.99      0.99     19783
weighted avg       0.99      0.99      0.99     19783



In [169]:
print(classification_report(val_labels, val_predictions4))

              precision    recall  f1-score   support

       Adult       0.67      0.87      0.76        75
      Benign       0.92      0.92      0.92       805
   Malicious       0.98      0.97      0.97      2057

    accuracy                           0.95      2937
   macro avg       0.86      0.92      0.88      2937
weighted avg       0.96      0.95      0.95      2937



# Aggregation model

In [156]:
train_pred_proba = model.predict_proba(train_features)
val_pred_proba = model.predict_proba(val_features)
train_pred_proba4 = model4.predict_proba(train_features4)
val_pred_proba4 = model4.predict_proba(val_features4)

train_features_final = np.concatenate([train_pred_proba, train_pred_proba4], axis=1)
val_features_final = np.concatenate([val_pred_proba, val_pred_proba4], axis=1)

In [157]:
model_final = SGDClassifier()
model_final.fit(train_features_final, train_labels);

In [158]:
train_predictions_final = model_final.predict(train_features_final)
val_predictions_final = model_final.predict(val_features_final)

In [159]:
print(classification_report(train_labels, train_predictions_final))

              precision    recall  f1-score   support

       Adult       1.00      1.00      1.00      4301
      Benign       1.00      1.00      1.00      4789
   Malicious       1.00      1.00      1.00     10693

    accuracy                           1.00     19783
   macro avg       1.00      1.00      1.00     19783
weighted avg       1.00      1.00      1.00     19783



In [160]:
print(classification_report(val_labels, val_predictions_final))

              precision    recall  f1-score   support

       Adult       0.88      0.92      0.90        75
      Benign       0.95      0.94      0.94       805
   Malicious       0.98      0.98      0.98      2057

    accuracy                           0.97      2937
   macro avg       0.94      0.95      0.94      2937
weighted avg       0.97      0.97      0.97      2937

